In [1]:
import pandas as pd
import numpy as np
from ML_pr import *
from sklearn.linear_model import LogisticRegression


In [2]:
crime_data_2021_present = pd.read_csv('csv_files/Crimes_2014.csv')

In [3]:
crime_data_2021_present

,ID,New_Date,Primary Type,Location Description,Arrest,Community Area,RegionName,Severity_Score
0,12045583,2020-05-07 10:24:00,THEFT,APARTMENT,0,35.0,Douglas,Low
1,12031001,2020-04-16 05:00:00,BATTERY,APARTMENT,1,60.0,Bridgeport,Medium
2,12093529,2020-07-01 10:16:00,ASSAULT,STREET,1,46.0,South Chicago,Medium
3,12178140,2020-09-27 23:29:00,BATTERY,RESIDENCE - PORCH / HALLWAY,0,67.0,West Englewood,Medium
4,12126129,2020-08-04 20:28:00,WEAPONS VIOLATION,STREET,1,71.0,Auburn Gresham,Medium
...,...,...,...,...,...,...,...,...
2510651,12131776,2020-08-10 20:30:00,BURGLARY,APARTMENT,0,66.0,Chicago Lawn,Medium
2510652,12082414,2020-06-18 21:00:00,THEFT,RESIDENCE,0,49.0,Roseland,Low
2510653,12118237,2020-07-27 15:02:00,BATTERY,APARTMENT,0,27.0,East Garfield Park,Medium
2510654,12142591,2020-08-14 15:00:00,MOTOR VEHICLE THEFT,STREET,0,2.0,West Ridge,Medium


ML - 1. Predict if a person would get arrested considering primary type of crime, location description and neighborhood (region name)


# Best Feature

<h5>Splitting the data into testing and training set. To be cautious about not touching the testing data, we saved it to a csv file</h5>

In [4]:
X = crime_data_2021_present[['Primary Type', 'Location Description', 'RegionName']]
y = crime_data_2021_present['Arrest'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # 80% training, 20% testing
X_test['Arrest'] = y_test
X_test.to_csv("csv_files/Crime_Testing_Dataset.csv")


<h5>Get the feature which will give us the best accuracy</h5>

In [6]:
(features, accuracy) = feature_selection_and_evaluation(X_train, y_train)

In [6]:
print(features, accuracy)

['Primary Type', 'Location Description', 'RegionName'] 0.8786626398290486


In [7]:
baselineClf = MajorityLabelClassifier()
baselineClf.fit(X_train, y_train)
predict_Y = baselineClf.predict(X_train)

sum = 0
for x, x2 in zip(y_train, predict_Y):
    if(x == x2):
        sum += 1

print(sum)
print(sum/len(y_train))

0
1623679
0.8083941242424786


# Training/Testing the model
<h5> using the best feature that we got from feature_selection_and_evaluation() function, train the LogisticRegression Model using the same training set as earlier to avoid making new one and test the model on Crime_Testing_Dataset.csv </h5>

In [8]:
# # Training and Testing Data Split
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# encoder = OneHotEncoder(handle_unknown='ignore')  # Handle unknown categories by ignoring them

# # Training: Encode features to make it easier for ML to train
# X_train_encoded = encoder.fit_transform(X_train[features])
# # X_test_encoded = encoder.fit_transform(X_train['Primary Type'])

# # Testing: Encode features to make it easier for ML to test
# X_test_encoded = encoder.transform(X_test[features])
# # X_test_encoded = encoder.transform(X_test['Primary Type'])

# # Create and train the logistic regression model
# model = LogisticRegression(random_state=42, solver='liblinear', penalty='l1', C=1.0)  
# model.fit(X_train_encoded, y_train)

# # Predict on the testing dataset
# y_pred_test = model.predict(X_test_encoded)

# # Evaluate the accuracy of the model
# accuracy = accuracy_score(y_test, y_pred_test)

# print("Accuracy:", accuracy)





Accuracy: 0.8786852859407487


In [1]:
encoder = OneHotEncoder(handle_unknown='ignore')  # Handle unknown categories by ignoring them

# Training: Encode features to make it easier for ML to train
X_train_encoded = encoder.fit_transform(X_train[features])
print(X_train_encoded)
# X_test_encoded = encoder.fit_transform(X_train['Primary Type'])

# Testing: Encode features to make it easier for ML to test
X_test_encoded = encoder.transform(X_test[features])

# Split the data into training and testing sets

# Train a logistic regression model
model = LogisticRegression(random_state=42, solver='liblinear')
model.fit(X_train_encoded, y_train)

# Predict probabilities of being arrested on the test set
probabilities = model.predict_proba(X_test_encoded)[:, 1]

# You can also directly predict the class labels if needed
# y_pred = model.predict(X_test)

# Evaluate the model (optional)
accuracy = accuracy_score(y_test, (probabilities > 0.5).astype(int))
print("Accuracy:", accuracy)


NameError: name 'OneHotEncoder' is not defined

In [8]:
probabilities_arrest_df = pd.DataFrame(probabilities)
probabilities_arrest_df.rename(columns={0:'Prob'}, inplace=True)

In [9]:
probabilities_arrest_df

,Prob
0,0.071569
1,0.112263
2,0.076977
3,0.054855
4,0.200903
...,...
502127,0.229490
502128,0.252273
502129,0.055513
502130,0.183500


In [10]:
X_test['Prob'] = probabilities

In [22]:
len(X_test)

502132

In [ ]:
# Highly Recommended for Final report ()
# 2 data (Crime and Housing)

# You want to get a mean house price of all the region in Housing data (181 columns and 1 row) 
# Rotate teh data (2 columns (RegionName, AvgPrice))
# Zone for different housing prices (Ex 10-50,000 Zone 1, 50-100,000 Zone 2, so on)
# Join them (Housing Data and Crime) on Region Name  => New Data (Save CSV)
# Create a ML Model Classification (SVM, NN)
# Using the Zone, Region Name, Primary Type, Location Description => Will the person be arrested or not (0,1)


In [ ]:
# Do T-Test to test our hypothesis
# Get pre covid mean (Group the crime by region and then get the mean crime in the Chicago)
# null hypothesis - The average crime in post covid is the average crime in pre covid (Fail to reject the null hypothesis)
# alter hypothesis - The average crime in post covid is greater than average crime in pre covid (Reject null hypothesis)

# 2nd Model Idea
make a model such that it uses crime and housing data to predict the avg house price in that neighborhood given total crime in that neighborhood. Use linear regression becuase since we are predicting prices. 